# Amazon Personalize Content Generator Workshop - Lab 9

In this lab we will create thematic descriptions for related items using the Personalize Content Generator. This capability of Personalize uses generative AI under-the-hood to produce a short and compelling description that captures the most impactful thematic elements for a set of items. The thematic descriptions can then be used as the title or label for a carousel or grid widget used to display related items or as part of the email subject when targeting users who may be interested in the items. These generated descriptions are more compelling alternatives than plain titles. For example, "Ultimate audio quality for all your listening needs" for a set of audio products rather than "Compare similar items" or "You may also like".

![Content Genetator](images/retaildemostore-content-generator-ex3.png)

The Content Generator uses a model trained with the [Similar-Items](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-similar-items.html) recipe to identify related items based on a seed item and then uses a large language model (LLM) to generate the thematic description. Although Similar-Items can be used with real-time inference or batch inference, themes can only be generated with batch inference jobs. In addition, using the Content Generator incurs additional cost (see [pricing page](https://aws.amazon.com/personalize/pricing/) for details). For this lab, we will generate themes the similar items for the featured products in the Retail Demo Store catalog.

You can read more about the Content Generator in the Personalize [documentation](https://docs.aws.amazon.com/personalize/latest/dg/personalize-with-gen-ai.html#gen-ai-themed-rec) or [blog](https://aws.amazon.com/about-aws/whats-new/2023/11/amazon-personalize-themes-generative-ai/).

## Setup

The workshop will be using the python programming language and the AWS SDK for python. Even if you are not fluent in python, the code cells should be reasonably intuitive. In practice, you can use any programming language supported by the AWS SDK to complete the same steps from this workshop in your application environment.

### Update dependencies

To get started, we need to perform a bit of setup. First, we need to ensure that a current version of botocore is locally installed. The botocore library is used by boto3, the AWS SDK library for python. We need a current version to be able to access some of the newer Amazon Personalize features.

The following cell will update pip and install the latest botocore library.

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade pip numexpr
!{sys.executable} -m pip install --upgrade --no-deps --force-reinstall botocore

### Import dependencies and prepare clients

First we will import the libraries and create the clients needed for this workshop.

In [ ]:
# Import dependencies
import boto3
import json
import pandas as pd
import numpy
import time
import requests
import botocore
from datetime import datetime
from IPython.display import display, HTML
from packaging import version

# Create clients
personalize = boto3.client('personalize')
servicediscovery = boto3.client('servicediscovery')
ssm = boto3.client('ssm')

The following cell will load the saved variables from the earlier foundational Personalize labs. The core personalization labs are required by this lab.

In [ ]:
%store -r

## Lookup working S3 bucket name

We will stage the segmentation job input file on S3 and have the segmentation job output written to S3. We'll use the same S3 stack bucket used for other Personalize workshops. The bucket name is stored in SSM so let's lookup the value.

In [ ]:
bucketresponse = ssm.get_parameter(
    Name='retaildemostore-stack-bucket'
)

# We will use this bucket to store our training data:
bucket = bucketresponse['Parameter']['Value']     # Do Not Change

print('Bucket: {}'.format(bucket))

## Retrieve IP address of Products microservice

We will use the featured products from the Retail Demo Store's catalog as the seed items to generate thematic descriptions. Let's get the local IP address of the Products microservice so we can call its API to retrieve products.

In [ ]:
response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='products',
    MaxResults=1,
    HealthStatus='HEALTHY'
)

assert len(response['Instances']) > 0, 'Products service instance not found; check ECS to ensure it launched cleanly'

products_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
print('Products Service Instance IP: {}'.format(products_service_instance))

## Load featured products into DataFrame

Let's load all of the products from the Products microservice into Pandas dataframe and then isolate the featured products into their own dataframe. We'll be using the featured products to generate the themes.

In [ ]:
response = requests.get('http://{}/products/all'.format(products_service_instance))
products = response.json()
products_df = pd.DataFrame(products)
pd.set_option('display.max_rows', 5)

# Isolate the featured products
featured_df = products_df[products_df["featured"] == 'true']
featured_df

## Prepare input file for batch inference job

Next we will prepare a batch inference input file that includes the feature product item IDs. The maximum number of items allowed in a theme generation batch inference job is 100 so we'll be sure to limit the number of items we include in the input file. Batch inference jobs that do not include theme generation have a limit of 50M items.

First, let's consider the format of the job input file. Below is a sample of the input file for a related items batch inference job for 3 items. The format is the same regardless of whether themes are being generated or not:

```javascript
{"itemId": "2"}
{"itemId": "4"}
{"itemId": "6"}
```

Notice that each item is represented on a single line as an independent JSON document. This is the [JSON Lines](https://jsonlines.org/) format. For our batch inference job, we will use the product IDs for featured products, being sure not to exceed the limit.

In [ ]:
featured_ids = featured_df['id'].tolist()[:100]
print(featured_ids)

In [ ]:
# Create and write job input file to disk
json_input_filename = "related_items_json_input.json"
with open(json_input_filename, 'w') as json_input:
    for id in featured_ids:
        json_input.write(f'{{"itemId": "{id}"}}\n')

Display the job input file contents. As noted above, one very important characteristic of the job input file is that the JSON document for each `itemId` must be fully defined on its own line.

In [ ]:
!cat $json_input_filename

## Upload job input file to S3 bucket

Before we can create a batch inference job to generate similar items and themes, we have to upload the job input file to our S3 bucket.

In [ ]:
# Upload file to S3
boto3.Session().resource('s3').Bucket(bucket).Object(json_input_filename).upload_file(json_input_filename)
s3_input_path = "s3://" + bucket + "/" + json_input_filename
print(s3_input_path)

## Define job output location

We also need to define an output location in our S3 bucket where the batch inference job writes its output.

In [ ]:
# Define the output path
s3_output_path = "s3://" + bucket + "/related-items/similar-items/"
print(s3_output_path)

## Create batch inference job with theme generation

Finally, we're ready to create a batch inference job that includes theme generation. There are several required parameters as well as optional parameters needed for theme generation.

- The solution version ARN for the Similar-Items model we created in a prior lab (see Lab 3). Theme generation is currently limited to solution versions trained with the Similar-Items recipe.
- The IAM role that Personalize needs to access the job input file and write the output file. This role was created by a CloudFormation template for this project.
- The filter ARN that ensures that similar items are from the same category as the seed item (created in Lab 4).
- A batch inference job mode of `THEME_GENERATION` that tells Personalize that we want themes created based on the the thematic similarity of related items for each seed item.
- The theme generation configuration that tells Personalize which field/column in the items dataset represents the item name. We created the `PRODUCT_NAME` field in the items dataset schema in Lab 2.
- Job input and output locations.
- Limit the number of similar items for each seed item to 15 items.

In [ ]:
response = personalize.create_batch_inference_job (
    solutionVersionArn = similar_items_solution_version_arn,
    jobName = "retaildemostore-related-items_" + str(round(time.time()*1000)),
    roleArn = role_arn,
    filterArn = include_category_filter_arn,
    batchInferenceJobMode = "THEME_GENERATION",
    themeGenerationConfig = {
      "fieldsForThemeGeneration": {
          "itemName": "PRODUCT_NAME"
      }
    },
    jobInput = {"s3DataSource": {"path": s3_input_path}},
    jobOutput = {"s3DataDestination":{"path": s3_output_path}},
    numResults = 15
)
job_arn = response['batchInferenceJobArn']
print(json.dumps(response, indent=2, default=str))

## Wait for batch inference job to complete

The batch inference job can take 20-30 minutes to complete. Even though our input file only specifies a few items, there is a certain amount of fixed overhead required for Personalize to provision the compute resources needed to execute the job. This overhead is amortized for larger input files that generate related items and themes for more items. The theme generation job type also requires more processing time to generate each theme.

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = job_arn
    )
    status = response["batchInferenceJob"]['status']
    print("BatchInferenceJob: {}".format(status))

    if status == "ACTIVE" or status == "CREATE FAILED":
        break

    time.sleep(60)

## Download and inspect job output file

Let's identify the output file in the output location in the S3 bucket, download the output file to the local volume, and display its contents.

In [ ]:
s3 = boto3.client("s3")

job_start_time = response["batchInferenceJob"]["creationDateTime"]

response = s3.list_objects_v2(
    Bucket=bucket,
    Prefix="related-items/similar-items/",
)

output_key_name = None

for obj in response["Contents"]:
    if obj["LastModified"] > job_start_time and obj["Key"].endswith(".out"):
        output_key_name = obj["Key"]
        break

assert output_key_name is not None, "Unable to locate the job output file in the output folder"

print(f"Downloading output file {output_key_name} from {bucket}")

out_file = json_input_filename + ".out"
s3.download_file(bucket, output_key_name, out_file)

print("Output file contents:")
!cat $out_file

Notice that the input seed item ID is echoed in the output file in the `input` element and we also have an `output` element for each seed item. The `output` element has a `recommendedItems` array that contains the related/similar item IDs for the seed item as well as a `theme` element and `itemsThemeRelevanceScores` array. The recommended items are ranked by the theme relevance score. The score is in a rough range of -0.1 and 0.6. The higher the score, the more closely related the item is to the theme. You might use the scores to set a threshold to show only items that are strongly related to the theme. The guidance is to use a threshold of 0.1 but you should evaluate the scores with your data. You can find the score for each recommended item in the `itemsThemeRelevanceScores` array where the index in the theme relevance array matches the index in the recommended items array.

## Inspect generated themes

Let's take a look at the generated themes for the similar items for each seed item.

The following cell will output details on each seed item including its generated theme and the theme relevancy score for each similar item.

After running the cell below, here are some aspects to inspect.
- Do the generated themes match the seed items?
- Do the similar items for the seed item match the seed item and generated theme? Be sure to focus on the product description rather than the product image since the product description and name are the key inputs to the theme generation process.
- For similar items that are not thematically similar to the seed item or theme, are the scores lower than items that are thematically similar? 

In [ ]:
pd.options.display.max_rows = 15

with open(out_file) as themes_file:
    # Read all lines from the segmentation output file.
    themes_lines = themes_file.readlines()

    for idx, theme_line in enumerate(themes_lines):
        theme_results = json.loads(theme_line)
        item_id = theme_results["input"]["itemId"]

        if not "output" in theme_results:
            if "error" in theme_results:
                display(HTML(f'<div class="alert alert-block alert-danger">Error generating similar items and theme for item {item_id}: {theme_results["error"]}</div><hr/>'))
            else:
                display(HTML(f'<div class="alert alert-block alert-danger">Unknown error generating similar items and theme for item {item_id}</div><hr/>'))

            continue

        similar_items = theme_results["output"]["recommendedItems"]
        theme = theme_results["output"]["theme"]
        theme_scores = theme_results["output"]["itemsThemeRelevanceScores"]

        item = featured_df.loc[featured_df["id"] == item_id].iloc[0]

        similar_items_df = pd.DataFrame()
        for si_idx, similar_item_id in enumerate(similar_items):
            similar_item = products_df.loc[products_df["id"] == similar_item_id].iloc[0]
            similar_item_info = {
                "image": [ '<img width="200" src="' + similar_item["image"] + '"/>' ],
                "item": [ f'<h4>{similar_item["name"]}</h4><i>{similar_item["description"]}</i><br/>{similar_item_id}' ],
                "category": [ similar_item["category"] ],
                "score": [ theme_scores[si_idx] ]
            }

            similar_items_df = pd.concat([similar_items_df, pd.DataFrame(data=similar_item_info)], axis=0, ignore_index=True)

        # Display details on all users in the segment
        display(HTML(f'<h2>Theme {idx + 1}: {theme}</h2>'))
        display(HTML(f'<h3>Seed item {idx + 1}: {item["name"]}</h3><p><i>{item["description"]}</i><br/>{item_id}</p>'))
        display(HTML(f'<img width="250" src="{item["image"]}"/>'))
        display(HTML(f'<h3>Similar items</h3>'))
        display(HTML(similar_items_df.to_html(escape=False)))
        display(HTML('<hr/>'))

## Update product catalog with themes

To put the generated themes to work in the Retail Demo Store web application, we'll update each of our featured products with the related items and generated theme via the Products service. This will allow the web application to display this information when it is available for a product.

The following cell iterates over the batch inference output file again but this time will construct a PUT REST API call to the Products service. Notice too that it only includes recommended items with a theme relevance score above a minimum threshold. The IDs of the related items and the generated theme are persisted in the `related_items` and `related_items_theme` fields, respectively.

In [ ]:
# We will limit related items to those with a score >= 0.1
score_threshold = 0.1

with open(out_file) as themes_file:
    # Read all lines from the segmentation output file.
    themes_lines = themes_file.readlines()

    for idx, theme_line in enumerate(themes_lines):
        theme_results = json.loads(theme_line)
        item_id = theme_results["input"]["itemId"]
        if not "output" in theme_results:
            if "error" in theme_results:
                print(f'Error generating similar items and theme for item {item_id}: {theme_results["error"]}')
            else:
                print(f'Unknown error generating similar items and theme for item {item_id}')

            continue

        similar_items = theme_results["output"]["recommendedItems"]
        theme = theme_results["output"]["theme"]
        theme_scores = theme_results["output"]["itemsThemeRelevanceScores"]

        final_items = []
        for idx_item, similar_item_id in enumerate(similar_items):
            score = theme_scores[idx_item]
            if score >= score_threshold:
                final_items.append(similar_item_id)

        response = requests.get(f"http://{products_service_instance}/products/id/{item_id}")
        product = response.json()

        if len(final_items) >= 3:
            product["related_items_theme"] = theme.rstrip(".")
            product["related_items"] = final_items
        else:
            print(f"Not enough items with score >= {score_threshold} for item {item_id}; clearing related items fields for product")
            product.pop("related_items_theme", None)
            product.pop("related_items", None)

        print(f"Updating related items for product {item_id}")
        headers = {"Content-Type": "application/json"}
        response = requests.put(f"http://{products_service_instance}/products/id/{item_id}", json=product, headers=headers)
        if not response.ok:
            print(f"status_code={response.status_code}")

print("Done updating products")

Now let's read back the details for the last product updated to inspect the two new fields, `related_items` and `related_items_theme`, we just added to the product.

In [ ]:
response = requests.get(f"http://{products_service_instance}/products/id/{item_id}")
product = response.json()
print(json.dumps(product, indent=2))

## Inspect thematic description and related items in web UI

We can see our generated theme and related items in action on the product detail page in the Retail Demo Store UI. This page retrieves similar items from the Recommendations microservice and displays them below the product details. The Recommendations microservice has logic that will check the product record for the presence of a pre-generated theme and related items and return them. Otherwise, it will call real-time Personalize endpoints for models trained with the Similar-Items and Personalized-Ranking recipes to obtain related items and rerank them for the current user.

Since we generated themes and related items for the featured products in the catalog, we just need to click on any product in the "Featured products" widget on the Retail Demo Store's homepage as shown below to get to the product detail page.

![Click featured product](images/retaildemostore-content-generator-ex1.png)

This takes us to the product detail page where we can see the generated theme and related items from the batch inference job we ran earlier.

![Content Generator](images/retaildemostore-content-generator-ex2.png)

For other products (i.e., not featured products), you will see the default similar items user experience. Additionally, if an A/B test is active for the product detail page, it will short-circuit the logic that displays the generated theme.

## Lab complete

Congratulations! You have completed the Personalize content generator lab.